In [ ]:
import json
import os
import json
import pandas as pd


In [ ]:
# COLLEGAMENTO CON GOOGLE DRIVE
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data è la lista dei dizionari
# claim data è un dizionario di dizionari
#claim id: 0
#claim structure:
#{'specifications': {'0': {'name': 'Dataset', 'value': 'Omniglot'}, '1': {'name': 'Model', 'value': 'MAML w/ LinNet'}},
#'Measure': 'adaptation learning rate', 'Outcome': '0.08'}



def find_names_values_in_json_files(directory_path,names_to_allign,values_to_allign):
    # Elenca tutti i file JSON nella cartella
    files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(".json")]

    # Dizionario per raccogliere i risultati
    alligned_name_positions = {}
    alligned_value_positions = {}
    measure_id = None

    for file_name in files:
        # Estrai paperID e tableID dal nome del file
        base_name = os.path.basename(file_name).split(".json")[0]
        paper_id, table_id, non_relevant= base_name.split("_")

        # Leggi il contenuto del file JSON
        with open(file_name, 'r') as f:
            data = json.load(f)

        # Itera attraverso gli elementi della lista (data è una lista di dizionari)
        for claim_data in data:
          for claim_id, claim in claim_data.items():
            # Ottieni il dizionario con le specifiche
            for spec, spec_data in claim.items():
                if spec == 'specifications':
                  for spec_id, name_value in spec_data.items():
                    name = name_value['name']
                    value = name_value['value']
                     # se il nome della specifica deve essere allineato
                    if name in names_to_allign.keys():
                        name_value['name'] = names_to_allign.get(name)
                        position = f"{paper_id}_{table_id}_{claim_id}_{spec_id}"
                        if name not in alligned_name_positions:
                          alligned_name_positions[name] = [] # aggiungi la chiave value
                        alligned_name_positions[name].append(position)

                    if value in values_to_allign.keys():
                        name_value['value'] = values_to_allign.get(name)
                        position = f"{paper_id}_{table_id}_{claim_id}_{spec_id}"
                        if value not in alligned_value_positions:
                          alligned_value_positions[value] = [] # aggiungi la chiave value
                        alligned_value_positions[value].append(position) # aggiungi la posizione di value nella lista delle posizioni
                    measure_id = int(spec_id) + 1 # l'id della measure sarà pari all'id dell'ultima specifica +1


                else:
                  if spec == 'Measure':
                    name = spec_data # in name ci va il nome della Measure
                    if name in names_to_allign.keys():
                        spec_data = names_to_allign.get(name)
                        position = f"{paper_id}_{table_id}_{claim_id}_{measure_id}"
                        if name not in alligned_name_positions:
                          alligned_name_positions[name] = [] # aggiungi la chiave value
                        alligned_name_positions[name].append(position)
                  if spec == 'Outcome':
                    value = spec_data # outcome. # in value ci va il valore della Measure
                    # Crea la posizione nel formato richiesto
                    if value in values_to_allign.keys():
                        spec_data = values_to_allign.get(value) # allineo il valore
                        position = f"{paper_id}_{table_id}_{claim_id}_{measure_id}" # l'id della Measure è uguale a quello della specifica + 1
                        if value not in alligned_value_positions:
                          alligned_value_positions[value] = [] # aggiungi la chiave value
                        alligned_value_positions[value].append(position)

        # Passo 3: Salvare le modifiche in una copia
        with open(f'/content/drive/My Drive/Ingegneria-dei-dati/alligned_claims/alligned_{base_name}.json', "w") as copia_file:
            json.dump(data, copia_file, indent=4)

    return alligned_name_positions,alligned_value_positions  # mappa di nome_allineato : posizione , mappa di valore_allineato : posizione



def save_alignment_in_json_file(alligned_name_positions, alligned_value_positions):
    print("Salvataggio dell'allineamento in corso...")
    # Creazione del JSON finale
    output_data = {
        "aligned_names": alligned_name_positions,
        "aligned_values": alligned_value_positions
    }

    # Scrittura su file JSON
    output_file = '/content/drive/My Drive/Ingegneria-dei-dati/claims_alignment.json'
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(output_data, file, indent=4, ensure_ascii=False)

    print("Scrittura del file json completata")







In [ ]:
# MAIN

names_to_allign = {
    "Method": ["Model"],
    "Algorithm": ["Model"],
    "Meta-Optimizer": ["Optimizer"],
    "Setting": ["Configuration"],
    "Training Setup": ["Configuration"],
    "Shots": ["K-shot"],
    "Shot": ["K-shot"],
    "accuracy": ["Accuracy"],
    "Accuracy(%)": ["Accuracy"],
    "accuracy(%)": ["Accuracy"],
    "Test Set o Training set": ["Dataset"],


}

values_to_allign = {
    "MiniImageNet": ["mini-ImageNet"],
    "miniImageNet": ["mini-ImageNet"],
    "MiniImagenet": ["mini-ImageNet"],
    "tieredImagenet": ["tieredImageNet"],
    "5w1s": ["5-way 1-shot"],
    "MAML w/": ["MAML"],
    "MAML (Finn et al., 2017)": ["MAML"],
    "ANIL w/": ["ANIL"],
    "ANIL (Raghu et al., 2019)": ["ANIL"]

}

claims_directory_path = '/content/drive/My Drive/Ingegneria-dei-dati/extracted_claims'
alligned_name_positions,alligned_value_positions = find_names_values_in_json_files(claims_directory_path,names_to_allign,values_to_allign)
save_alignment_in_json_file(alligned_name_positions,alligned_value_positions)








Salvataggio dell'allineamento in corso...
Scrittura del file json completata
